# Data preparation for predicting match results from past performance with a rolling window

Dependencies

In [66]:
import pandas as pd
from datetime import datetime
import numpy as np

**Data**: the dataset is downloaded from kaggle: https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017 
From 3 files, results.csv is used here.

In [67]:
df_raw = pd.read_csv("../data/raw/results.csv")
df_raw.info() ; df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47399 entries, 0 to 47398
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        47399 non-null  object
 1   home_team   47399 non-null  object
 2   away_team   47399 non-null  object
 3   home_score  47399 non-null  int64 
 4   away_score  47399 non-null  int64 
 5   tournament  47399 non-null  object
 6   city        47399 non-null  object
 7   country     47399 non-null  object
 8   neutral     47399 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 2.9+ MB


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


We take the year 1975 as a cut off, when Copa América was re-established after a hiatus. After this, the international football tournament structure has been relatively stable. To take into account a maximum of 2 years before, we limit to the time frame to begin in 1973.

In [68]:
df_raw["date"] = pd.to_datetime(df_raw["date"], format = "%Y-%m-%d")
df_recent = df_raw[df_raw["date"] >= datetime(1973, 1, 1)].copy()
df_recent.info() ; df_recent.head()

<class 'pandas.core.frame.DataFrame'>
Index: 38299 entries, 9100 to 47398
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        38299 non-null  datetime64[ns]
 1   home_team   38299 non-null  object        
 2   away_team   38299 non-null  object        
 3   home_score  38299 non-null  int64         
 4   away_score  38299 non-null  int64         
 5   tournament  38299 non-null  object        
 6   city        38299 non-null  object        
 7   country     38299 non-null  object        
 8   neutral     38299 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(5)
memory usage: 2.7+ MB


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
9100,1973-01-01,Gambia,Senegal,2,1,Friendly,Banjul,Gambia,False
9101,1973-01-08,Nigeria,Ghana,4,2,All-African Games,Lagos,Nigeria,False
9102,1973-01-08,Algeria,Tanzania,4,2,All-African Games,Lagos,Nigeria,True
9103,1973-01-08,Congo,Burkina Faso,3,0,All-African Games,Ibadan,Nigeria,True
9104,1973-01-08,Guinea,Egypt,4,1,All-African Games,Ibadan,Nigeria,True


## Preparing data for rolling window aggregations

To use the 'rolling' method of 'pandas', we set the date as index. Since dates are not unique for each match, we also create a match_id column. 

In [69]:
df_recent = df_recent.set_index("date").sort_index()
df_recent["match_id"] = np.arange(1, df_recent.shape[0] + 1, 1)

For simplicity, we will not distinguish between home and away performance, and transform the data into a team-level dataset.

In [70]:
df_hometeams = df_recent.drop(columns = "away_team")\
    .rename(columns = {
        "home_team" : "team",
        "home_score" : "scored",
        "away_score" : "conceded"
    }).copy()
df_awayteams = df_recent.drop(columns = "home_team")\
    .rename(columns = {
        "away_team" : "team",
        "away_score" : "scored",
        "home_score" : "conceded"
    }).copy()
df_teams = pd.concat([df_hometeams, df_awayteams], axis = 0)\
    .sort_values("match_id")

We create win and draw columns.

In [71]:
df_teams["win"] = (df_teams["scored"] > df_teams["conceded"]).astype(int)
df_teams["draw"] = (df_teams["scored"] == df_teams["conceded"].astype(int))

## Calculating performance metrics

We calculate performance metrics (win and draw ratios, average goals scored and conceded) in rolling windows of 6, 12, 18, and 24 months (defined in terms of days).

In [72]:
dict_perf_dfs = {}
for window in ["183D", "365D", "548D", "730D"]:
    # stats calculated without the current row
    df_stats = df_teams.groupby("team").rolling(window, closed = "left").agg({
        "win" : "mean",
        "draw" : "mean",
        "scored" : "mean",
        "conceded" : "mean"
    }).reset_index().rename(columns = { 
        # pandas named aggregation doesn't work with rolling, hence separate renaming
        "win" : f"win_ratio_roll{window}", 
        "draw" : f"draw_ratio_roll{window}", 
        "scored" : f"avg_goals_scored_roll{window}", 
        "conceded" : f"avg_goals_conceded_roll{window}"
    })
    # match_id of the current row
    df_matchid = df_teams.groupby("team").rolling(window).agg({
         "match_id" : "max"
    }).reset_index()
    dict_perf_dfs[f"df_perf_roll{window}"] = pd.concat([df_stats, df_matchid[["match_id"]]], axis = 1)


In [73]:
dict_perf_dfs["df_perf_roll183D"]

,team,date,win_ratio_roll183D,draw_ratio_roll183D,avg_goals_scored_roll183D,avg_goals_conceded_roll183D,match_id
0,Abkhazia,2012-09-25,NaN,NaN,NaN,NaN,27115.0
1,Abkhazia,2012-10-21,0.000000,1.000000,1.000000,1.000000,27258.0
2,Abkhazia,2014-06-01,NaN,NaN,NaN,NaN,28628.0
3,Abkhazia,2014-06-02,0.000000,1.000000,1.000000,1.000000,28633.0
4,Abkhazia,2014-06-04,0.500000,0.500000,1.500000,1.000000,28659.0
...,...,...,...,...,...,...,...
76593,Åland Islands,2017-06-29,0.333333,0.666667,1.000000,0.666667,31646.0
76594,Åland Islands,2023-07-09,NaN,NaN,NaN,NaN,37075.0
76595,Åland Islands,2023-07-10,0.000000,0.000000,0.000000,2.000000,37085.0
76596,Åland Islands,2023-07-11,0.000000,0.000000,0.500000,3.000000,37094.0


Then combine these into a single dataframe.

In [74]:
df_perf_rolling = dict_perf_dfs["df_perf_roll183D"]\
    .merge(dict_perf_dfs["df_perf_roll365D"], on = ["team", "date", "match_id"])\
    .merge(dict_perf_dfs["df_perf_roll548D"], on = ["team", "date", "match_id"])\
    .merge(dict_perf_dfs["df_perf_roll730D"], on = ["team", "date", "match_id"])\
    .dropna()
df_perf_rolling["match_id"] = df_perf_rolling["match_id"].astype(int)

## Adding performance data to match data

We will add the performance data for home and away teams, and we should name these columns separately. First, we create dictionaries to map column names to home and away teams.

In [75]:
home_mapper = {}
away_mapper = {}

for col_name in df_perf_rolling.columns.drop(["team", "date", "match_id"]):
    home_mapper[col_name] = f"home_{col_name}"
    away_mapper[col_name] = f"away_{col_name}"

In [76]:
df_match_perf = df_recent.loc["1975-01-01" : ]\
    .reset_index()\
    .merge(df_perf_rolling, left_on = ["match_id", "date", "home_team"], right_on = ["match_id", "date", "team"])\
    .rename(columns = home_mapper)\
    .drop(columns = "team")\
    .merge(df_perf_rolling, left_on = ["match_id", "date", "away_team"], right_on = ["match_id", "date", "team"])\
    .rename(columns = away_mapper)\
    .drop(columns = "team")

Better to have home advantage as binary than boolean neutrality.

In [77]:
df_match_perf["host_advantage"] = df_match_perf["neutral"] * (-1) + 1

Finally, we add the match result.

In [78]:
def get_match_result(row):
    if row["home_score"] > row["away_score"]:
        return "home_win"
    elif row["home_score"] < row["away_score"]:
        return "away_win"
    else:
        return "draw"
    
df_match_perf["result"] = df_match_perf.apply(get_match_result, axis = 1)

## Inspecting class balance

Here we check if there is a tendency to put winning teams as home teams for games played in neutral venues.

In [79]:
df_match_perf[df_match_perf["neutral"]].value_counts("result", normalize = True)

result
home_win    0.421380
away_win    0.336011
draw        0.242609
Name: proportion, dtype: float64

To reduce potential bias, we can take a random half of neutral games, and reverse the order of home and away teams. To swap the column names, below is a dictionary.

In [80]:
reverse_mapper = {}

for col_name in df_match_perf.columns:
    if "home" in col_name:
        reverse_mapper[col_name] = col_name.replace("home", "away")
    elif "away" in col_name:
        reverse_mapper[col_name] = col_name.replace("away", "home")

In [81]:
df_nonneutral = df_match_perf[~df_match_perf["neutral"]].copy()
df_neutral = df_match_perf[df_match_perf["neutral"]].sample(frac = 1, random_state = 42).copy()

half_len = len(df_neutral) // 2
df_neutral_keep = df_neutral.iloc[:half_len, :].copy()
df_neutral_reverse = df_neutral.iloc[half_len:, :].copy()

df_neutral_reverse = df_neutral_reverse.rename(columns = reverse_mapper)
df_neutral_reverse["result"] = df_neutral_reverse.apply(get_match_result, axis = 1)

Recombine and check the balance again.

In [82]:
df_match_perf_balanced = pd.concat([df_nonneutral, df_neutral_keep, df_neutral_reverse], axis = 0).sort_values("date").reset_index(drop = True)
df_match_perf_balanced[df_match_perf_balanced["neutral"]].value_counts("result", normalize = True)

result
away_win    0.378963
home_win    0.378428
draw        0.242609
Name: proportion, dtype: float64

## Saving the final dataframe

In [83]:
df_match_perf_balanced.to_pickle("../data/processed/rolling_performance.pkl")